In [9]:
import pandas as pd
import bertinizer as bz

In [3]:
delay_df = pd.read_excel("/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/deploy_ml/get_around_delay_analysis.xlsx")
pricing_df = pd.read_csv("/Users/antoinebertin/Documents/jedha/full_stack/projects_full_stack/deploy_ml/get_around_pricing_project.csv")

In [49]:
delay_df.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [20]:
pricing_df.drop('Unnamed: 0', axis=1, inplace=True)

In [53]:
(pricing_df["has_getaround_connect"] == True).sum()

2230

In [55]:
pricing_df.dtypes

model_key                    object
mileage                       int64
engine_power                  int64
fuel                         object
paint_color                  object
car_type                     object
private_parking_available      bool
has_gps                        bool
has_air_conditioning           bool
automatic_car                  bool
has_getaround_connect          bool
has_speed_regulator            bool
winter_tires                   bool
rental_price_per_day          int64
dtype: object

# questions

- Which share of our owner’s revenue would potentially be affected by the feature?
- How many rentals would be affected by the feature depending on the threshold and scope we choose?
- How often are drivers late for the next check-in? How does it impact the next driver?
- How many problematic cases will it solve depending on the chosen threshold and scope?

In [40]:
#threshold: how long should the minimum delay be?
#scope: should we enable the feature for all cars?, only Connect cars?
threshold = 60
scope_getaround_connect = True

In [42]:
# Filter out cancellations as they do not have a delay
rental_data_filtered = delay_df[delay_df['state'] == 'ended']
# Calculate the percentage of rentals that were late
late_rentals_percentage = (rental_data_filtered['delay_at_checkout_in_minutes'] > 0).mean() * 100
# Calculate the average delay for rentals that were late
average_delay = rental_data_filtered[rental_data_filtered['delay_at_checkout_in_minutes'] > 0]['delay_at_checkout_in_minutes'].mean()
late_rentals_percentage, average_delay

(52.11415904682738, 201.79147171416417)

In [43]:
# Filter out rentals with delay at checkout in minutes greater than or equal to threshold
rentals_with_delay_threshold = rental_data_filtered[rental_data_filtered['delay_at_checkout_in_minutes'] >= threshold]

# Get unique car_id from rentals_with_delay_threshold
affected_cars = rentals_with_delay_threshold['car_id'].unique()

# Calculate the percentage of cars that would be affected
affected_cars_percentage = len(affected_cars) / rental_data_filtered['car_id'].nunique() * 100

# Print the affected cars and the percentage
print(f"Cars affected by a threshold set to {threshold}: {affected_cars}")
print(f"Percentage of cars affected by a threshold set to {threshold}: {affected_cars_percentage}%")

Cars affected by a threshold set to 60: [359049 402596 398816 ... 376462 395485 406927]
Percentage of cars affected by a threshold set to 60: 42.660735948241005%
